<a href="https://colab.research.google.com/github/Naimish240/SPH-Star-Formation/blob/main/SPH_Article.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Star Formation Simulation in Python**
---
By : [Naimish Mani B](https://www.github.com/naimish240)

President, Stellaria SASTRA.

---


## **Table of Contents**

1. Introduction
2. Fundamentals
  - Star Formation
  - Smooth Particle Hydrodynamics
  - Governing Equations
3. Setup
  - Initial Conditions and Constants
  - Kernel Function
  - Density Calculation
  - Pressure Calculation
  - Acceleration Calculation
  - Plot Frame
4. Simulation
  - Main Loop
  - Animation
5. Conclusions and Future Work
6. Resources

## **1. Introduction**

So in this article, we'll be looking at how to write a star-formation simulation in Python. Sure, the topic may sound intimidating on first glance, but it's actually pretty simple and straightforward to implement. All you need to know are some basic programming in Python 3 and the fundamentals of vector algebra. An understanding of calculus is a plus, but isn't absolutely essential. We won't be deriving equations, but rather using them directly without proof. This article aims to be as beginner-friendly as possible. If you get stuck somewhere, feel free to reach out to me for clarifications.

<br>

First things first. What exactly _is_ a simulation? A computer simulation is basically just a fancy way of saying that we're using computers to imititate a real-world process. Here, the real world process we're interested in is that of star formation. In order to imitate the real-world process, we either take a rule-based approach or solve our governing equations. Given that star formation is an actively-studied topic in astrophysics with plenty of equations describing the same, we will be taking the latter approach.

<br>

Unlike us, computers can't really sit and work out the solutions to an equation on their own. So, we use different "schemes" to solve our governing equations. Here, we'll be using a technique known as "Smooth Particle Hydrodynamics" in order to solve our equations and run the simulation. The equations are in their vector form.

<br>

Given that we're using Python to solve our equations, we use NumPy to speed up the vector calculations and MatPlotLib to view the simulation results. I will be linking tutorials for the same at the end of the article, and would strongly suggest you check them out to get a better understanding for how these libraries work.

## **2. Fundamentals**

Before we jump the metaphorical gun and dive head-first into the code, we first need to understand some of the basic concepts that are being used here. Firstly, we'll look at the physics of how stars form, followed up by the simulation technique we'll be using to visualise the same.

### **2.1 Star Formation**

Through our observations of the night sky, we know that there's a lot of gas in outer space, just floating around. We also know that stars burn this gas to produce light, through nuclear fusion. What we're trying to figure out, is how a gas cloud goes from just floating around in space, to collapsing on itself and starting nuclear fusion.

Broadly speaking, there are 5 major steps in the formation of a star as identified by astronomers. They are as follows:

1. There's a cloud of molecular gas which doesn't collapse on itself by virtue of the motion of its particles. The gas just has pressure, no gravity.

2. Something (we don't know what, yet) triggers the gas to form lumps, which in turn leads to gravity dominating over the pre existing pressure.

3. These clumps of gas merge together and gain mass, slowly forming protostars, with a positive feedback loop where the more clumps joins together, the heavier it gets, attracting more clumps.

4. As the clumps start aggregating, the gas glob starts rotating, eventually flattening out (due to it's moment of inertia) and formimg the circumstellar disk. This disk contains two parts, the accretion disk and the protoplanetary disk.

5. The accretion disk gets pressurized till the gas starts a fusion reaction. This initial fusion reaction sends out a strong wind through the protoplanetary disc, helping the remaining matter aggregate to form planets and asteroids, while the fusion reaction starts heating up the gas and forming a stable star.


Our simulation will look at a dark cloud, and study how the gravitational collapse might occur. Subsequent steps will require an in-depth understanding of thermodynamics and magnetohydrodynamics, among other things, and are hence beyond the scope of this article. Resouces to learn more about this can be found in the resources section.



<img src="https://www.researchgate.net/profile/Markus-Wittkowski/publication/314283237/figure/fig8/AS:654789746647058@1533125449614/The-different-phases-of-star-formation.png">

### **2.2 Smooth Particle Hydrodynamics**

As the name suggests, Smooth Particle Hydrodynamics (SPH) takes a particle-based approach to modelling the properties of a fluid. In our case, we model the clumps of gas as homogenous through interpolation and analytical differentiation. This makes it very simple to apply our standard energy and momentum conservation equations on each "particle". This is in contrast to traditional fluid dynamics solvers, which make use of meshing and the finite volume approach.

In effect, we study the flow of a fluid by treating it as a collection of particles. We will be looking at how SPH can be implemented in the upcoming sections. For now, this bird's eye view of SPH is enough to get a feel for what we will be doing.

<img src="https://www.researchgate.net/publication/324097218/figure/fig3/AS:609695752523782@1522374203410/Schematic-for-the-symmetric-particle-based-method-Smoothed-Particle-Hydrodynamics-SPH.png">



### **2.3 Governing Equations**

For the SPH simulation, we represent the fluid as a collection of particles. Each particle has an initial position, velocity and mass, with both position and velocity being vectors.


Let velocity be $v$ and position be $r$. Then,

According to Euler's equation of continuity, for an ideal fluid,

$\frac{1}{\rho}\frac{dv}{dt} = -\nabla P + f$
---
Where,

$P$ : Pressure of the fluid

$\rho$ : Density of the fluid

$f$ : External forces acting on the system.

We assume $P$ to be depicted through a polytropic process, which has the equation

$P = k \rho^{1+\frac{1}{n}}$
---
Where,

$k$ : Normalization constant

$n$ : Polytropic index


Similarly, we assume the external force $f$ to be given by the equation

$f = -\lambda r - \nu v$
---
Where $\lambda r$ denotes the force due to gravity and $\nu v$ the force due to viscousity acting on the fluid.

These equations have been adopted from [this](https://academic.oup.com/mnras/article/350/4/1449/986477) paper


## **3. Setup**

Now that we've understood the basics of star formation and SPH, we can start making the solver. First, lets import the libraries we'll be using.

In [1]:
import numpy as np
import matplotlib.pyplot as plt

### **3.1 Initial Conditions and Constants**

Every simulation requires a set of initial conditions to kick things off. I.e., the state of the system at $t_0$. We are defining our initial conditions here. Some of these constants might not make sense just yet, but they will make sense as we move along.

Initially, we start the system with particles (lumps of gas) spread out randomly. 

In [41]:
# Constants

N     = 500                   # Number of particles
t     = 0                     # start time of simulation
tEnd  = 8                     # end time for simulation
dt    = 0.01                  # timestep
M     = 2                     # star mass
R     = 0.75                  # star radius
h     = 0.1                   # smoothing length
k     = 0.1                   # equation of state constant
n     = 1                     # polytropic index
nu    = 1                     # damping
m     = M/N                   # single particle mass
lmbda = 2.01                  # lambda for gravity
Nt    = int(np.ceil(tEnd/dt)) # number of timesteps

In [3]:
# Initial Conditions
def initial():
  np.random.seed(42)             # set random number generator seed
  pos   = np.random.randn(N,2)   # randomly selected positions and velocities
  vel   = np.zeros(pos.shape)
  return pos, vel

### **3.2 Kernel Function**

As we know, SPH makes use of a smoothing function to distribute the mass inside each particle in space. This kernel function is derived from the Dirac $\delta$ function. It is essential that the kernel approximates this function at its limits to ensure the rest of the maths checks out. There are many standard smoothing kernels, like the Gaussian kernel, Cubic spline kernel, Quintic spline kernel, etc. Check [this](https://pysph.readthedocs.io/en/latest/reference/kernels.html) for more info on kernels.

Here, we make use of the Gaussian Kernel, which is defined as follows:

$W(r, h) = \frac{1}{h^3 \pi^{1.5}} \exp({\frac{- {\lVert{r}\rVert}^2}{h^2}})$
---

Similarly, we will also be requiring the gradient of the gaussian kernel, which is defined as follows:

$\nabla W(r, h) = \frac{-2}{h^5 \pi^{1.5}}\exp({\frac{- {\lVert{r}\rVert}^2}{h^2}})r$
---

The derivation of the gradient is left to the reader as an exercise.

Both of these functions $W$ and $\nabla W$ will be used extensively in the following calculations, because discretiziation helps us shift the gradient onto the kernel. Check [this](https://interactivecomputergraphics.github.io/SPH-Tutorial/slides/01_intro_foundations_neighborhood.pdf) link for more info.

In [4]:
# Gaussian Smoothing Kernel
def kernel(x, y, h):
  """
  Input:
    x : matrix of x positions
    y : matrix of y positions
    h : smoothing length

  Output:
    w : evaluated smoothing function
  """
  # Calculate |r|
  r = np.sqrt(x**2 + y**2)
  # Calculate the value of the smoothing function
  w = (1.0 / (h*np.sqrt(np.pi)))**3 * np.exp( -r**2 / h**2)
  # Return
  return w

In [5]:
# Smoothing Kernel Gradient
def gradKernel(x, y, h):
  """
  Inputs:
    x : matrix of x positions
    y : matrix of y positions
    h : smoothing length

  Outputs:
    wx, wy : the evaluated gradient
  """
  # Calculate |r|
  r = np.sqrt(x**2 + y**2)
  # Calculate the scalar part of the gradient
  n = -2 * np.exp( -r**2 / h**2) / h**5 / (np.pi)**(3/2)
  # Get the vector parts of the gradient
  wx = n * x
  wy = n * y
  # Return the gradient vector
  return wx, wy

### **3.3 Density Calculation**

In SPH, the density at a particular point can be found through the formula

$\rho_i = \sum_{j} mW(r_i - r_j, h)$
---

From this formula, we can see that we need to find the distance between all particles to get the density at a particular position. Hence, we can get the density at every position as shown below.

In [6]:
# Solve for the (r_i - r_j) term in the density formula
def magnitude(ri, rj):
  """
  Inputs:
    ri : M x 2 matrix of positions
    rj : N x 2 matrix of positions
  
  Output:
    dx, dy : M x N matrix of separations
  """
  
  M = ri.shape[0]
  N = rj.shape[0]
  
  # get x, y of r_i
  r_i_x = ri[:,0].reshape((M,1))
  r_i_y = ri[:,1].reshape((M,1))
  
  # get x, y of r_j
  r_j_x = rj[:,0].reshape((N,1))
  r_j_y = rj[:,1].reshape((N,1))
  
  # get r_i - r_j
  dx = r_i_x - r_j_x.T
  dy = r_i_y - r_j_y.T
  
  return dx, dy

In [7]:
# Get density at sample points
def density(r, pos, m, h):
  """
  Inputs:
    r   : M x 3 matrix of sampling locations
    pos : N x 3 matrix of particle positions
    m   : particle mass
    h   : smoothing length

  Output:
    rho : M x 1 vector of densities
  """
  
  M = r.shape[0]
  dx, dy = magnitude(r, pos);
  rho = np.sum(m * kernel(dx, dy, h), 1).reshape((M,1))
  
  return rho

### **3.4 Pressure Calculation**

Since we already have the $\rho$ values from the previous step, we can directly calculate the pressure at each point using the formula

$P = k \rho^{1+\frac{1}{n}}$
---
as we had defined in 2.3.

In [8]:
# Find pressure
def pressure(rho, k, n):
  """
  Inputs:
    rho : vector of densities
    k   : equation of state constant
    n   : polytropic index

  Output:
    P : pressure
  """

  P = k * rho**(1+1/n)
  
  return P

### **3.5 Acceleration Calculation**

In order to track the motion of the particles, we need to know their positions at all time steps. But in order to know the position, we need to know the velocity. And to know the velocity, we require the acceleration of each particle. We can calculate the acceleration acting on each particle at every time step using the following discretization formula:

$\frac{dv_i}{dt} = - \sum_{i, j \ne 0} m(\frac{P_i}{\rho_i^2}+\frac{P_j}{\rho_j^2}) \nabla W(r_i - r_j, h) + f_i$
---

where 'i' denotes the ith particle and
$f_i = -\lambda r_i - \nu v_i$
---

from section 2.3.

Although this formula might look complicated, we can clearly see that it's just a sum of the components which we had written functions for earlier. Hence, by daisy-chaining the functions, we can evaluate this expression easily.

In [9]:
# Calculate acceleration on particles
def acceleration(pos, vel, m, h, k, n, lmbda, nu):
  """
  Inputs:
    pos   : N x 2 matrix of positions
    vel   : N x 2 matrix of velocities
    m     : particle mass
    h     : smoothing length
    k     : equation of state constant
    n     : polytropic index
    lmbda : external force constant
    nu    : viscosity

  Output:
    a : N x 2 matrix of accelerations
  """
  
  N = pos.shape[0]
  
  # Calculate densities
  rho = density(pos, pos, m, h)
  
  # Get pressures
  P = pressure(rho, k, n)
  
  # Get pairwise distances and gradients
  dx, dy = magnitude(pos, pos)
  dWx, dWy = gradKernel(dx, dy, h)
  
  # Add Pressure contribution to accelerations
  ax = - np.sum(m * (P/rho**2 + P.T/rho.T**2) * dWx, 1).reshape((N,1))
  ay = - np.sum(m * (P/rho**2 + P.T/rho.T**2) * dWy, 1).reshape((N,1))
  
  # Pack acceleration components
  a = np.hstack((ax,ay))
  
  # Add external forces
  a += -lmbda * pos - nu * vel
  
  # Return total acceleration
  return a

## **4. Simulation**

Now that the setup is complete, we can move ahead and run the simulation. The simulation requires two main parts. Firstly, we need to set the main loop to run the simulation forward in time. Next, we need to stitch the frames together to form an animation.

### **4.1 Main Loop**

Here, we set up the loop that controls the simulation in the time domain. By tracking the particles in space through time, we can finally tie the simulation together in this loop. Here, we update the velocity and position using Newton's laws of motion.

$v_i = v_{i-1} + a_i t$
---
$r_i = r_{i-1} + v_i t$
---

In [42]:
import os
import glob
import tqdm

# Creates folder if it doesn't exist
if not os.path.exists('output'):
  os.mkdir('output')
# Else, deletes all images inside it
else:
  files = glob.glob('output/*.png')
  for f in files:
      os.remove(f)

In [43]:
# Disable inline printing to prevent all graphs from being shown
%matplotlib
pos, vel = initial()

# Start loop
for i in tqdm.tqdm(range(Nt)):
  # Update values
  acc = acceleration(pos, vel, m, h, k, n, lmbda, nu)
  vel += acc * dt
  pos += vel * dt
  rho = density(pos, pos, m, h)

  # Plot
  fig, ax = plt.subplots(figsize=(6, 6))

  plt.sca(ax)
  plt.cla()

  # Get color from density
  cval = np.minimum((rho-3)/3,1).flatten()

  # Place particles on map with color
  plt.scatter(pos[:,0],pos[:,1], c=cval, cmap=plt.cm.autumn, s=5, alpha=0.75)

  # Set plot limits and stuff
  ax.set(xlim=(-2.5, 2.5), ylim=(-2.5, 2.5))
  ax.set_aspect('equal', 'box')
  ax.set_facecolor('black')
  ax.set_facecolor((.1,.1,.1))

  # save plot
  plt.savefig(f'output/{i}.png')
  plt.close()

Using matplotlib backend: agg


100%|██████████| 800/800 [02:22<00:00,  5.60it/s]


### **4.2 Animation**

Now that all the frames are ready, we can stitch them together into a video using OpenCV, and download the simulation.

In [44]:
import cv2

img_array = []

print("Reading Frames")
# List image paths
imgs_list = glob.glob('output/*.png')
lsorted = sorted(imgs_list, key=lambda x: int(os.path.splitext(x[7:])[0]))

for filename in tqdm.tqdm(lsorted):
    img = cv2.imread(filename)
    height, width, layers = img.shape
    size = (width,height)
    img_array.append(img)

out = cv2.VideoWriter('simulation.mp4',cv2.VideoWriter_fourcc(*'MP4V'), 30, size)

# Write to video
print("Writing frames")
for i in tqdm.tqdm(range(len(img_array))):
    out.write(img_array[i])
out.release()

Reading Frames


100%|██████████| 800/800 [00:02<00:00, 336.58it/s]


Writing frames


100%|██████████| 800/800 [00:00<00:00, 860.53it/s]


In [45]:
# Download the simulation
from google.colab import files
files.download('simulation.mp4')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

## **5. Conclusion and Future Work**

Through this exercise, we learnt about how star formation works, and how we can use SPH to simulate the same, albeit in a very crude, 2D case. In the real world, scientists take their simulation results and compare it with observational data, and tweak their SPH model assumptions accordingly. Now all that's left is to go find observational data and tweak our models!

Also, don't forget to clear the output folder before you re-run simulations!

## **6. Resources**

Introduction to Simulations Video : https://www.youtube.com/watch?v=eED4bSkYCB8

NumPy Tutorial : https://www.youtube.com/watch?v=QUT1VHiLmmI

MatplotLib Playlist : https://www.youtube.com/watch?v=UO98lJQ3QGI&list=PL-osiE80TeTvipOqomVEeZ1HRrcEvtZB_

SPH Workshop Slides : https://interactivecomputergraphics.github.io/SPH-Tutorial/slides/01_intro_foundations_neighborhood.pdf

SPH Kernels : https://pysph.readthedocs.io/en/latest/reference/kernels.html

SPH Review Paper : https://doi.org/10.1007/s11831-010-9040-7

SPH Workshop Notes : https://interactivecomputergraphics.github.io/SPH-Tutorial/

Toy Stars in 1D paper : https://academic.oup.com/mnras/article/350/4/1449/986477

MATLAB Toy star implementation : https://pmocz.github.io/manuscripts/pmocz_sph.pdf

PySPH conference : https://www.youtube.com/watch?v=l-f16KjR9Bw

SPH Mathematics and proofs : https://www.youtube.com/watch?v=tAXHCAEgSuE

Another SPH review paper : https://doi.org/10.1146/annurev.aa.30.090192.002551

2D toy stars : https://academic.oup.com/mnras/article/365/3/991/969452

Star Formation article : http://web.gps.caltech.edu/~gab/astrophysics/astrophysics.html

Star Formation video : https://www.youtube.com/watch?v=lI57XIZ17hE&t=1s